# Simularium Conversion Tutorial : CytoSim Data

In [1]:
from IPython.display import Image
import numpy as np
from simulariumio import Converter

This notebook provides example python code for converting your own simulation trajectories into the format consumed by the Simularium Viewer. It creates a .simularium JSON file which you can drag and drop onto the viewer like this:

![title](img/drag_drop.gif)

***
## Prepare your spatial data

The Simularium custom data Converter consumes spatiotemporal data from CytoSim. It requires the following data:
* **box_size** : *np.ndarray (shape = [3])*
    * A numpy ndarray containing the XYZ dimensions of the simulation bounding volume
* **fiber_points_path** : *str*
    * filepath to the Cytosim output file fiber_points.txt
    
Optionally, you can also include:
* **plots** : *Dict[str, Any]*
    * An object containing plot data already in Simularium format

In [2]:
box_size = 500.

example_data = {
    "box_size" : np.array([box_size, box_size, box_size]),
    "fiber_points_path" : "data/cytosim/fiber_points.txt"
}

## Convert and save as .simularium JSON file

Once your data is shaped like in the `example_data` object, you can use the converter to generate the file at the given path:

In [3]:
Converter(example_data, "cytosim").write_JSON("/Users/blairl/Desktop/cytosim_example")

IndexError: index -1 is out of bounds for axis 0 with size 0

## Visualize in the Simularium viewer

In your browser, either Firefox or Chrome, navigate to https://staging.agentviz.allencell.org/ and drag your file onto the center viewer window. 

*For now you'll first have to choose an example trajectory and close the load window. Once the example trajectory loads, you can drop your own file in to replace it. We'll fix this soon :)*